In [1]:
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

camera = CSICamera(width=224, height=224, capture_fps=40)

from jetracer.nvidia_racecar import NvidiaRacecar

car = NvidiaRacecar()

In [2]:
import cv2
import ipywidgets
import threading
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
import traitlets
from IPython.display import display
from ipywidgets import Layout, Button, Box
import ipywidgets.widgets as widgets
from jetcam.utils import bgr8_to_jpeg
import cv2
prediction_widget = ipywidgets.Image(format='jpeg', width=500, height=500)
camera.running = True
import re
A, B, C, D, E, F = None, None, None, None, None, None
qr_detected = False  

# Function to extract variables from QR code value and save them as global variables
def extract_variables_from_qr_code(value):
    global A, B, C, D, E, F
    # Using regular expression to find all occurrences of letter followed by digits
    matches = re.findall(r'([A-Za-z])(\d+)', value)
    
    # Iterating through matches and saving them as global variables
    for match in matches:
        variable_name = match[0]
        variable_value = match[0] + match[1]
        globals()[variable_name] = variable_value
# Global variable to control thread execution
qr_reader_running = True

printed_qr_codes = set()

def read_qr_code_from_camera():
    """Read QR code from camera feed."""
    global qr_reader_running, qr_detected
    global A, B, C, D, E, F
    try:
        detect = cv2.QRCodeDetector()

        while qr_reader_running:
            frame = camera.value  # Read a frame from the camera
            prediction = frame.copy()
            
            value, bbox, straight_qrcode = detect.detectAndDecode(frame)

            if value and value not in printed_qr_codes:
                print("QR Code Value:", value)
                printed_qr_codes.add(value)
                extract_variables_from_qr_code(value)
                if A is not None:
                    print("A:", A)
                if B is not None:
                    print("B:", B)
                if C is not None:
                    print("C:", C)
                if D is not None:
                    print("D:", D)
                if E is not None:
                    print("E:", E)
                if F is not None:
                    print("F:", F)
                if bbox is not None and len(bbox) > 0:
                    x1, y1 = bbox[0][0][0], bbox[0][0][1]
                    x2, y2 = bbox[2][0][0], bbox[2][0][1]
                 
                    prediction = cv2.rectangle(prediction, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    prediction = cv2.putText(prediction, value, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

                prediction_widget.value = bgr8_to_jpeg(prediction)
                qr_detected = True  # Set QR detected flag
                qr_reader_running = False  # Stop recording when QR code is detected
                print(f"QR Detected: {qr_detected}")
            else:
                prediction_widget.value = bgr8_to_jpeg(prediction)
    except Exception as e:
        print("An error occurred:", str(e))
        return None

# Function to stop the QR code reader thread
def stop_qr_code_reader():
    global qr_reader_running
    qr_reader_running = False

# Function to run read_qr_code_from_camera() in a separate thread
def run_qr_code_reader():
    global qr_reader_thread
    qr_reader_thread = threading.Thread(target=read_qr_code_from_camera)
    qr_reader_thread.start()

# Call the function to start the QR code reader in a separate thread


# Call the function to stop the QR code reader thread (wherever you want to stop it)
# stop_qr_code_reader()

In [3]:
#!/usr/bin/env python
from flask import Flask, render_template, Response
import cv2
import threading


app = Flask(__name__)

server_running = True
# After gen is called infinite loop will take frame by frame from camera class,
# and yeild it to the client in jpeg data form
def gen():
    """Video streaming generator function."""
    while server_running:
        frame = camera.value
        if frame is not None:
            # Resize the frame to be twice as big
            # frame = cv2.resize(frame, (frame.shape[1] * 2, frame.shape[0] * 2))
            
            # Encode the resized frame into JPEG format
            frame_bytes = cv2.imencode('.jpg', frame)[1].tobytes()
            yield (b'--frame\r\n'
                   b'Content-Type: image/jpeg\r\n\r\n' + frame_bytes + b'\r\n')
        else:
            print("Error: Empty frame received.")

# When client is forwarded to /video_feed, it will call 'gen' function
# with imported camera class as a paremeter
@app.route('/video_feed')
def video_feed():
    """Video streaming route. Put this in the src attribute of an img tag."""
    return Response(gen(), mimetype='multipart/x-mixed-replace; boundary=frame')

# When client demands root, index.html will forward it to video feed
@app.route('/')
def index():
    """Seurveillance camera monitoring"""
    return render_template('index.html')

@app.route('/one')
def hello_one():
    return 'hello one'

@app.route('/image')
def image():
    frame = camera.value
    if frame is not None:
        frame_bytes = cv2.imencode('.jpg', frame)[1].tobytes()
        return Response(frame_bytes, mimetype='image/jpeg')
    else:
        # Handle case when the frame is empty
        return "Error: Empty frame received."

def run_flask():
    app.run(host='0.0.0.0', port=5000, threaded=True, debug=False, use_reloader=False)

if __name__ == '__main__':
    flask_thread = threading.Thread(target=run_flask)
    flask_thread.start()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production


In [4]:
import socket
import time
import threading

running = False

def handle_client(clientsocket):
    global running
    while True:
        try:
            message = clientsocket.recv(1024).decode()  # Receives Message
            if not message:
                break
            message = message.strip()
            if message == "control_on":
                running = True
                print("Control mode activated.")
            elif message == "control_off":
                running = False
                print("Control mode deactivated.")
            elif running:  # Only execute control commands if control_mode is True
                if message == "forward":
                    car.throttle = -0.5
                elif message == "left":
                    car.steering = 1.0
                elif message == "right":
                    car.steering = -1.0
                elif message == "forward_left":
                    car.throttle = -0.5
                    car.steering = 1.0
                elif message == "forward_right":
                    car.throttle = -0.5
                    car.steering = -1.0
                elif message == "reverse":
                    car.throttle = 0.5
                elif message == "reverse_left":
                    car.throttle = 0.5
                    car.steering = 1.0
                elif message == "reverse_right":
                    car.throttle = 0.5
                    car.steering = -1.0
                elif message == "stop":
                    car.throttle = 0.0
                    car.steering = 0.0
        except ConnectionResetError:
            break

def start_server():
    # Defines Server Values
    listensocket = socket.socket()
    Port = 5001
    maxConnections = 999
    IP = socket.gethostname() # Gets Hostname Of Current Machine
    ip_address = socket.gethostbyname(IP)

    listensocket.bind(('', Port))

    # Opens Server
    listensocket.listen(maxConnections)
    print("Server started at " + IP + " on port " + str(Port))
    print("IP Address:" + ip_address)

    # Accepts Incoming Connections in a loop
    while True:
        try:
            (clientsocket, address) = listensocket.accept()
            print("New connection made!")
            client_thread = threading.Thread(target=handle_client, args=(clientsocket,))
            client_thread.start()
        except Exception as e:
            print(f"Error accepting connections: {e}")
            break

server_thread = threading.Thread(target=start_server)
server_thread.start()

   Use a production WSGI server instead.
Server started at nano-4gb-jp451 on port 5001
IP Address:127.0.1.1
 * Debug mode: off


In [5]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('road_following_model2-Copy1.pth'))

 * Running on all addresses.
 * Running on http://192.168.1.14:5000/ (Press CTRL+C to quit)


<All keys matched successfully>

In [6]:
from utils import preprocess

STEERING_GAIN = -2
STEERING_BIAS = -0.3

In [7]:
import torchvision
from torch2trt import TRTModule

#model = TRTModule()
#ALEXNET
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 8)
model.load_state_dict(torch.load('recognition_model.pth'))
device = torch.device('cuda')
model = model.to(device)

In [8]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
import time

#image = widgets.Image(format='jpeg', width=224, height=224)
#camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

blocked_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Blocked probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

free_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Free probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)


fid1_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid1 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

fid2_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid2 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)
fid3_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid3 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

fid4_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid4 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

fid5_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid5 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

fid6_disp = widgets.FloatProgress(
    value=0.5,
    min=0,
    max=1.0,
    description='Fid6 probability:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

steering_disp = widgets.FloatProgress(
    value=0.0,
    min=-1.0,
    max=1.0,
    description='Direction:',
    bar_style='info',
    style={'bar_color': '#ffff00'},
    orientation='horizontal'
)

fps = widgets.FloatText(
    value=0.0,
    description='FPS:',
    disabled=False
)

obj = widgets.FloatText(
    value=1,
    description='Objetivo:',
    disabled=False
)

In [10]:
import serial
import time
import threading

arduino_port = '/dev/ttyACM0' 
baud_rate = 9600  
ser = serial.Serial(arduino_port, baud_rate, timeout=1)
time.sleep(2)


In [11]:
import socket
import threading
from pyfcm import FCMNotification

server_running = True

def run_socket_server():
    HOST = '0.0.0.0'  # Listen on all network interfaces
    PORT = 5002

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((HOST, PORT))
        server_socket.listen()

        print(f"Server listening on port {PORT}")

        while server_running:
            client_socket, addr = server_socket.accept()
            print(f"Connected to {addr}")

            with client_socket:
                while True:
                    data = client_socket.recv(1024)
                    if not data:
                        break
                    message = data.decode()
                    print(f"Received: {message}")
                    
                    if message.startswith("token="):
                        message_to_send = "Token received\n"
                        client_socket.sendall(message_to_send.encode())
                        print("Token received message sent to client")
                    else:
                        message_to_send = "Token not received\n"
                        client_socket.sendall(message_to_send.encode())
                        print("Message sent")

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
display(blocked_disp,free_disp,fid1_disp,fid2_disp,fid3_disp,fid4_disp,fid5_disp,fid6_disp,steering_disp,fps,obj)
import torch.nn.functional as F
import queue

t_dir = 0.0
n_dir = 0.0
t_tot = 0.0
n_tot = 0
fps_tot = 0
t_tot_1 = time.time() 

objetivo = 1
obj.value = objetivo

# Hàng đợi để lưu trữ dữ liệu từ Arduino
data_queue = queue.Queue()

# Hàm đọc dữ liệu từ Arduino và đưa vào hàng đợi
def read_from_serial():
    while True:
        if ser.in_waiting > 0:
            line = ser.readline().decode('utf-8').rstrip()
            data_queue.put(line)
            
# Tạo luồng để đọc dữ liệu từ Arduino
serial_thread = threading.Thread(target=read_from_serial)
serial_thread.daemon = True
serial_thread.start()

def get_next_serial_line():
    if not data_queue.empty():
        return data_queue.get()
    return "Null"
def clear_queue():
    while not data_queue.empty():
        try:
            data_queue.get_nowait()
        except queue.Empty:
            break
            
# Initialize counter for consecutive detections
fid1_consecutive_count = 0
run_qr_code_reader()
display(prediction_widget)
while True:
    if not running:
        if qr_detected: 
            t_tot = time.time()
            fps_tot = 1/(t_tot-t_tot_1)
            t_tot_1 = t_tot
            fps.value = fps_tot
            for i in range(3):
                x = camera.value
            x = preprocess(x)
            y = model(x)


            # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
            y = F.softmax(y, dim=1)

            prob_blocked = float(y.flatten()[0])
            blocked_disp.value = prob_blocked
            prob_fid1 = float(y.flatten()[1])
            fid1_disp.value = prob_fid1
            prob_fid2 = float(y.flatten()[2])
            fid2_disp.value = prob_fid2
            prob_fid3 = float(y.flatten()[3])
            fid3_disp.value = prob_fid3
            prob_fid4 = float(y.flatten()[4])
            fid4_disp.value = prob_fid4
            prob_fid5 = float(y.flatten()[5])
            fid5_disp.value = prob_fid5
            prob_fid6 = float(y.flatten()[6])
            fid6_disp.value = prob_fid6
            prob_free = float(y.flatten()[7])
            free_disp.value = prob_free


            prob_max = float(max(y.flatten()))
            dirrect = "right"
            line =  get_next_serial_line()
            if not line == "Null":
                print(line)
            if prob_blocked == prob_max:
                fid1_consecutive_count = 0
                car.throttle = 0.0
            elif line == "Detect Obstacles":
                fid1_consecutive_count = 0
                push_service = FCMNotification(api_key="AAAAR4SAIBs:APA91bE8hxTnaAuQFdx2PmOnXsNBZhKwFoSEX1b3Gr6YULkt7tTh9rMBaXB-EONyWTbIWI3KGnjiIGFCpSB7nPt6EYwHPGhopQ2rRAKFOoxM4_p1xN8eZU42bGlb5jYeUCtx06Y79Hl7")
                #registration_id = "dZuIIHBhS1yEBpaeumyJ3G:APA91bGLiqPBTl7U8ommmhXAEaStJsjNu4pXpzbf4iXq0CYHOJWmeRUYAtk6YOgR2GIs9KqMQWSq5XyXW_T2zUnKn5PgPKDkgxtk2iu7uSoXr2RGZuulwS6KELJ-3MZV2b8UB__4jytn"
                registration_id = "fdacs6KnQHauVXFj7GqkkA:APA91bFNsQcVwRGbHGIJAoHCXwEk1kRpj-9lY4rvDlb-F94xNy1i65Wkgw6D3OKcOU419jSTVJIMcXwDxPjtzUbHblhX88rSKK0oAAaTB6YiH5zgNpUfMsjY9lhKtNwbqpwTjue6W5Na"
                message_title = "Jetracer"
                message_body = "Detect Obstacles!!! Please process."
                result = push_service.notify_single_device(registration_id=registration_id, message_title=message_title, message_body=message_body)
                while line != "Free":
                    car.throttle = 0.0
                    line = get_next_serial_line()
                    if running == True:
                        break
            elif prob_fid1 == prob_max and objetivo == 1:
                fid1_consecutive_count += 1
                if fid1_consecutive_count >= 2:
                    clear_queue()
                    ser.write(b'1')
                    if A == "A2":
                        while line != "Complete":
                            car.throttle = -0.25
                            car.steering = -1
                            line = get_next_serial_line()
                            if not line == "Null":
                                print(line)
                        ser.write(b'2')
                        clear_queue()
                    elif dirrect == "left":
                        while line != "Complete":
                            car.throttle = -0.25
                            car.steering = 1
                            line = get_next_serial_line()
                            if not line == "Null":
                                print(line)
                        ser.write(b'2')
                    clear_queue()
            else:
                fid1_consecutive_count = 0
                car.throttle = -0.25
                for i in range(3):
                    x = camera.value
                x = preprocess(x)
                x = x.half()
                output = model_trt(x).detach().cpu().numpy().flatten()
                x1 = float(output[0])
                car.steering = x1 * STEERING_GAIN + STEERING_BIAS
                steering_disp.value = x1
        else:
            pass
    else:
        pass

FloatProgress(value=0.5, bar_style='info', description='Blocked probability:', max=1.0, style=ProgressStyle(ba…

FloatProgress(value=0.5, bar_style='info', description='Free probability:', max=1.0, style=ProgressStyle(bar_c…

FloatProgress(value=0.5, bar_style='info', description='Fid1 probability:', max=1.0, style=ProgressStyle(bar_c…

FloatProgress(value=0.5, bar_style='info', description='Fid2 probability:', max=1.0, style=ProgressStyle(bar_c…

FloatProgress(value=0.5, bar_style='info', description='Fid3 probability:', max=1.0, style=ProgressStyle(bar_c…

...

192.168.1.6 - - [13/Jun/2024 09:23:42] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.6 - - [13/Jun/2024 09:24:41] "GET / HTTP/1.1" 200 -


New connection made!
Control mode activated.


192.168.1.6 - - [13/Jun/2024 09:24:41] "GET /video_feed HTTP/1.1" 200 -
192.168.1.6 - - [13/Jun/2024 09:24:42] "GET /favicon.ico HTTP/1.1" 404 -
192.168.1.6 - - [13/Jun/2024 09:35:52] "GET / HTTP/1.1" 200 -


New connection made!
Control mode activated.


192.168.1.6 - - [13/Jun/2024 09:35:52] "GET /video_feed HTTP/1.1" 200 -
192.168.1.6 - - [13/Jun/2024 09:35:52] "GET /favicon.ico HTTP/1.1" 404 -
